In [ ]:
%pip -q install torch
%pip -q install transformers
%pip -q install einops

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

sysprompt = "The assistant gives helpful, detailed, and polite answers to the user's questions.\n"

model = AutoModelForCausalLM.from_pretrained("teknium/Puffin-Phi-v2", trust_remote_code=True, torch_dtype=torch.bfloat16).to(device)
tokenizer = AutoTokenizer.from_pretrained("teknium/Puffin-Phi-v2", trust_remote_code=True, torch_dtype=torch.bfloat16)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

input_text = f"{sysprompt}User: Write an essay discussing the historical, cultural, and environmental significance of the Alps, focusing on their role in shaping the development of Europe and the world.\n Assistant:"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

outputs = model.generate(input_ids, max_length=300, do_sample=True, temperature=0.2, top_p=0.9, use_cache=True, repetition_penalty=1.2, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

Ooof.... COT is diffcult with this one...

In [ ]:
input_text = f"""{sysprompt}
User: Which is a faster way to get home?\n
User: Option 1-Take an 10 minutes bus, then an 40 minute bus, and finally a 10 minute train.\n
User: Option 2-Take a 90 minutes train, then a 45 minute bike ride, and finally a 10 minute bus.\n
Assistant: Option 1 will take 10+40+10 = 60 minutes.\n
Assistant: Option 2 will take 90+45+10 = 145 minutes.\n
Assistant: Since Option 1 takes 60 minutes and Option 2 takes 145 minutes, Option 1 is less than option 2 which makes it faster.\n

User: Which is a faster way to get to work?\n
User: Option 1 - Take a 1000 minute bus, then a half hour train, and finally a 10 minute bike ride.\n
User: Option 2 - Take an 800 minute bus, then an hour train, and finally a 30 minute bike ride.\n
User: Take a deep breath and think step by step. Which option is faster?\n
Assistant:
"""
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

outputs = model.generate(input_ids, max_length=3000, do_sample=True, temperature=0.1, eos_token_id=tokenizer.eos_token_id)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)